


</div>
<table>
    <tr style="background-color:white;"> 
        <td> <img src="files/Python_logo.png" style="width:50px;height:50px;float:right"> </td>
        <td> <p style="float:right;color:Green;font-size:40px">Data Analysis Practice</p></td>
    </tr>
</table>    

<b>Question:</b> Is the referral/marketing campaign feasible and earning the profit?
<br><b>Scenario:</b> Customers who signs up are given discount of <b>10 dollars</b> if they share certain marketing/referral text on their social media. Someone who signs up using that referral link on social media shared status will get discount worth <b>20 dollars</b>.


<br>Given Information:</br> Company has been collecting data on this campaign and has some data for Users and referrals in these two tables:


<br>

* Table User
    - id, name, referring_user_id
* Table Purchase
    - id, user_id, date, total, discounts


_______________

<b>Appraoch:</b> My approach here is to create the dummy data for both the tables using python and then use SQL(as in company settings generally SQL is used to fetch the data from databases) to fetch the data and provide the answers:

Importing all the important packages that might be required for the case study:

In [12]:
import pandas as pd
import numpy as np
import random
import string
from random import shuffle, seed
from faker.providers.person.en import Provider
import pandasql
import datetime
import time
import warnings
warnings.filterwarnings("ignore")

## Function for generating ids:

In [13]:
def id_generator(n, j):
    x = []
    for i in range(j):
        x.append(''.join(random.choices(string.ascii_letters + string.digits, k=n)))
    return x

It takes input/argument <b>'n'</b> = length of desired id (I have chosen 10 over here, you'll find it in the below part of the code
as you proceed), and <b>'j'</b> total number of rows required in the data frame (I am going with 1000 users as of now, but can be varied depending on use case)

## Function for generating Names (First+Last):

In [14]:
def name_generator(j):
    first_names = list(set(Provider.first_names))
    last_names = list(set(Provider.last_names))
    seed(4321)
    shuffle(first_names)
    shuffle(last_names)
    x = []
    for i in range(j):
        x.append(random.choice(first_names)+' '+random.choice(last_names))
    return x

I have used already available package named <I><b>'faker'</I></b> which offers list of first and last names that can be used in various permuation and combination to generate different names. Function takes <b>'j'</b> as an argument i.e. number of rows as in the above function

## Function for generating Random Dates:

In [15]:
def random_dates(start, end, n):
    start_u = start.value//10**9
    end_u = end.value//10**9
    x = pd.Series(pd.to_datetime(np.random.randint(start_u, end_u, n), unit='s'))
    x = x.map(lambda x: x.date())
    return x

Above function takes start and end dates and generates random dates between those 2 dates. It also takes <b>'n'</b> as an argument for total number of rows required to output

### Starting with creating the User table that has :  id, name, referring_user_id

In [16]:
df = pd.DataFrame(
        columns = ['id','name','referring_user_id']) ### Initiating the dataframe
df

,id,name,referring_user_id


In [17]:
df['id'] = pd.Series(id_generator(10, 1000)) ### Generating the id feature
df.head(5)

,id,name,referring_user_id
0,pvtSfPMlj9,NaN,NaN
1,TRI8NMbhlU,NaN,NaN
2,id5V3LSuSy,NaN,NaN
3,KtXpdiZBGq,NaN,NaN
4,DEMrm37Qur,NaN,NaN


In [18]:
df['name'] = pd.Series(name_generator(1000)) ### Generating the names
df.head(5)

,id,name,referring_user_id
0,pvtSfPMlj9,Julisa O'Reilly,NaN
1,TRI8NMbhlU,Azzie Marquardt,NaN
2,id5V3LSuSy,Bird Hudson,NaN
3,KtXpdiZBGq,Danny King,NaN
4,DEMrm37Qur,Stacie Hagenes,NaN


In [19]:
df['referring_user_id'] = np.random.permutation(
            df.id.sample(int(df.shape[0]))) ### Generating the referring user id
df.loc[df.sample(
        frac=0.7).index, 'referring_user_id'] = pd.np.nan ### Inserting 
                    ##some Null values on random as 
#all the users won't be referred
df.head(5)

,id,name,referring_user_id
0,pvtSfPMlj9,Julisa O'Reilly,NaN
1,TRI8NMbhlU,Azzie Marquardt,NaN
2,id5V3LSuSy,Bird Hudson,CQcNWn1ZSb
3,KtXpdiZBGq,Danny King,NaN
4,DEMrm37Qur,Stacie Hagenes,tBhSeCdSKf


In [20]:
df ### This is how the table looks like, it has (1000,3) shape

,id,name,referring_user_id
0,pvtSfPMlj9,Julisa O'Reilly,NaN
1,TRI8NMbhlU,Azzie Marquardt,NaN
2,id5V3LSuSy,Bird Hudson,CQcNWn1ZSb
3,KtXpdiZBGq,Danny King,NaN
4,DEMrm37Qur,Stacie Hagenes,tBhSeCdSKf
...,...,...,...
995,XqhtS7oEuK,Octave Bergstrom,NaN
996,CGuEIlKo8l,Kati Berge,NaN
997,PengrVycXL,Bruce Moore,NaN
998,WFA38miUzU,Tyrone Waters,NaN


### Creating the Purchase table that has :  id, user_id, date, total, discounts

In [21]:
 ### Initiating the purchase table
df_purchase = pd.DataFrame(columns =
                        ['id', 'user_id','date','total','discount'])
df_purchase.head(5)

,id,user_id,date,total,discount


Generating the <b>id</b> feature and I have chosen <b>100,000</b> transaction/ purchases at random, just an assumption

In [22]:
df_purchase['id'] = pd.Series(id_generator(9, 100000)) ### Generating the id feature
df_purchase.head(5)

,id,user_id,date,total,discount
0,pvtSfPMlj,NaN,NaN,NaN,NaN
1,9TRI8NMbh,NaN,NaN,NaN,NaN
2,lUid5V3LS,NaN,NaN,NaN,NaN
3,uSyKtXpdi,NaN,NaN,NaN,NaN
4,ZBGqDEMrm,NaN,NaN,NaN,NaN


Taking all the ids from user table and
creating <b>'user_id'</b> feature in purchase table as we would like to join and see some results. I have used <b>.sample()</b>
method with <b>replcement = True</b> which would take values from ids and keep on replacing it meaning <I>duplicate</I> them 

In [23]:
df_purchase['user_id'] = df.id.sample(frac = 100, replace = True).values 
df_purchase.head(5)

,id,user_id,date,total,discount
0,pvtSfPMlj,vytX8t6RI0,NaN,NaN,NaN
1,9TRI8NMbh,zPUi3e2o1D,NaN,NaN,NaN
2,lUid5V3LS,YTfpPba7jb,NaN,NaN,NaN
3,uSyKtXpdi,LNhmdN4Z11,NaN,NaN,NaN
4,ZBGqDEMrm,fCkS3vrejy,NaN,NaN,NaN


Generating random dates starting from <b>2020 Jan 01 </b> to <b> 2020 Aug 01</b>, those dates are totally chosen at random and has no significance

In [24]:
df_purchase['date'] = random_dates(pd.to_datetime('2019-01-01'), 
                                   pd.to_datetime('2020-08-01'), n = 100000)
df_purchase.head(5)

,id,user_id,date,total,discount
0,pvtSfPMlj,vytX8t6RI0,2020-05-25,NaN,NaN
1,9TRI8NMbh,zPUi3e2o1D,2019-09-29,NaN,NaN
2,lUid5V3LS,YTfpPba7jb,2020-04-21,NaN,NaN
3,uSyKtXpdi,LNhmdN4Z11,2019-01-31,NaN,NaN
4,ZBGqDEMrm,fCkS3vrejy,2020-03-13,NaN,NaN


Generating float values for <b>'total'</b> feature with highest value = <b>400</b> and lowest value = <b>40</b>, just an assumption

In [25]:
df_purchase['total'] = round(pd.Series(np.random.uniform(low=40, high=400,
                                                     size=(10000,1)).tolist()).str[0],2)
df_purchase.head(5)

,id,user_id,date,total,discount
0,pvtSfPMlj,vytX8t6RI0,2020-05-25,173.03,NaN
1,9TRI8NMbh,zPUi3e2o1D,2019-09-29,227.72,NaN
2,lUid5V3LS,YTfpPba7jb,2020-04-21,215.89,NaN
3,uSyKtXpdi,LNhmdN4Z11,2019-01-31,244.31,NaN
4,ZBGqDEMrm,fCkS3vrejy,2020-03-13,344.28,NaN


Generating float values for <b>'discount'</b> feature with highest value = <b>100</b> and lowest value = <b>0</b>, again just an assumption. 

Also I have randomly imputed <b>60%</b> of the discount values with <b>0</b> because not everyone gets the discount.
Assumption: <b>40%</b> of the transacation has discounts attached to it


In [26]:
df_purchase['discount'] = round(pd.Series(np.random.randint(low=0, high=100, 
                                                    size=(10000,1)).tolist()).str[0],2)
df_purchase.loc[df_purchase.sample(frac=0.6).index, 'discount'] = 0
df_purchase.head(5)

,id,user_id,date,total,discount
0,pvtSfPMlj,vytX8t6RI0,2020-05-25,173.03,0.0
1,9TRI8NMbh,zPUi3e2o1D,2019-09-29,227.72,10.0
2,lUid5V3LS,YTfpPba7jb,2020-04-21,215.89,0.0
3,uSyKtXpdi,LNhmdN4Z11,2019-01-31,244.31,92.0
4,ZBGqDEMrm,fCkS3vrejy,2020-03-13,344.28,0.0


In [27]:
df_purchase ### This is how the purchase table looks like, it has (100000,5) shape

,id,user_id,date,total,discount
0,pvtSfPMlj,vytX8t6RI0,2020-05-25,173.03,0.0
1,9TRI8NMbh,zPUi3e2o1D,2019-09-29,227.72,10.0
2,lUid5V3LS,YTfpPba7jb,2020-04-21,215.89,0.0
3,uSyKtXpdi,LNhmdN4Z11,2019-01-31,244.31,92.0
4,ZBGqDEMrm,fCkS3vrejy,2020-03-13,344.28,0.0
...,...,...,...,...,...
99995,FWLP1OXeu,vNk8zKH5ut,2020-05-04,NaN,0.0
99996,CmmuEdjWf,VndO5JT8OI,2019-05-30,NaN,0.0
99997,QdpaUBc7F,q57b8nC4go,2019-08-26,NaN,NaN
99998,R88p16h3H,6HOqPwYp5W,2019-12-26,NaN,0.0


### Will use those 2 tables with random values and use SQL to fetch the data and performing analysis:

<b>Logic for referral 360 table:</b> The idea behind the referral table is to create a table that has basically the <b>names of referring_user_id</b> by joining the user table with with itself on a.referring_user_id = b.id (inner join table). 



Second step is to create the <b>2 flags</b>

<b>1. Referral_10_flag:</b> This flag is for people who posted the video on social media and will get <b>10 dollars</b> off on their total (assuming that discount is not already included in purchase table's discount feature).

<b>2. Referral_20_flag:</b> This flag is for people who clicked on the referral link posted by the people on social media, these people would get <b>20 dollars</b> off on their total.

These 2 flags are created by joining the inner table again with user table on a.id = b.reffering_user_id and using case when statements

In [28]:
user_table_referral_360 = pandasql.sqldf("""
                        Select a.*,
                               b.id,
                               b.name as referred_person,
                        case when a.referring_user_id <> '' then 1 else 0 
                        end as referral_20_flag,
                        case when b.name <> '' then 1 else 0
                        end as referral_10_flag
                        from 
                        (Select x.*, 
                                y.name as referring_person
                                from df x 
                                LEFT JOIN df y
                                on x.referring_user_id = y.id) a
                        LEFT JOIN df b
                        on a.id = b.referring_user_id;""")
user_table_referral_360

,id,name,referring_user_id,referring_person,id,referred_person,referral_20_flag,referral_10_flag
0,pvtSfPMlj9,Julisa O'Reilly,None,None,None,None,0,0
1,TRI8NMbhlU,Azzie Marquardt,None,None,None,None,0,0
2,id5V3LSuSy,Bird Hudson,CQcNWn1ZSb,Ovila Schaden,0e3b7pbORt,Huldah O'Conner,1,1
3,KtXpdiZBGq,Danny King,None,None,None,None,0,0
4,DEMrm37Qur,Stacie Hagenes,tBhSeCdSKf,Madeleine Jacobs,None,None,1,0
...,...,...,...,...,...,...,...,...
995,XqhtS7oEuK,Octave Bergstrom,None,None,None,None,0,0
996,CGuEIlKo8l,Kati Berge,None,None,ek3TtXvrNH,Lourdes Hauck,0,1
997,PengrVycXL,Bruce Moore,None,None,None,None,0,0
998,WFA38miUzU,Tyrone Waters,None,None,None,None,0,0


Creating a <b>master_purchase</b> table by joining the user_table_referral_360 table with purchase table on a.user_id = b.id and finally <b>adding 30 dollars</b> to the users who has both the flags <b>Referral_10_flag and Referral_20_flag</b> with value = <b>1</b>. And adding <b>20 dollars for Referral_20_flag = 1 and adding 10 dollars for Referral_10_flag = 1</b>

In [29]:
master_purchase = pandasql.sqldf("""
                      Select a.*,
                             b.referral_20_flag,
                             b.referral_10_flag,
                             case when b.referral_20_flag = 1 and b.referral_10_flag = 1 
                             then a.discount + 30
                             when b.referral_20_flag = 1 then a.discount + 20 
                             when b.referral_10_flag = 1 then a.discount + 10
                             else a.discount end as new_discount
                      from df_purchase a
                      LEFT JOIN user_table_referral_360 b
                      on a.user_id = b.id""")
master_purchase

,id,user_id,date,total,discount,referral_20_flag,referral_10_flag,new_discount
0,pvtSfPMlj,vytX8t6RI0,2020-05-25,173.03,0.0,NaN,NaN,0.0
1,9TRI8NMbh,zPUi3e2o1D,2019-09-29,227.72,10.0,NaN,NaN,10.0
2,lUid5V3LS,YTfpPba7jb,2020-04-21,215.89,0.0,NaN,NaN,0.0
3,uSyKtXpdi,LNhmdN4Z11,2019-01-31,244.31,92.0,NaN,NaN,92.0
4,ZBGqDEMrm,fCkS3vrejy,2020-03-13,344.28,0.0,1.0,1.0,30.0
...,...,...,...,...,...,...,...,...
99995,FWLP1OXeu,vNk8zKH5ut,2020-05-04,NaN,0.0,NaN,NaN,0.0
99996,CmmuEdjWf,VndO5JT8OI,2019-05-30,NaN,0.0,0.0,1.0,10.0
99997,QdpaUBc7F,q57b8nC4go,2019-08-26,NaN,NaN,NaN,NaN,NaN
99998,R88p16h3H,6HOqPwYp5W,2019-12-26,NaN,0.0,1.0,1.0,30.0


Finally creating the profit and loss table by calculating the <b>revenue = total - new_discount</b> and finally taking the sum of <b>revenue</b>. If that revenue is positive, it means that after removing all the discounts, the campaign is still making positive revenue and should continue with the campaign. If the revenue is negative then the campaign should stop as the discounts sent are exceeding then the revenue being generated with that referral campaign

In [30]:
profit_loss = pandasql.sqldf("""
                             select sum(Revenue) as Total_Revenue from 
                             (select a.*, 
                             (total-discount) as Revenue 
                             from master_purchase a)""")
profit_loss

,Total_Revenue
0,1976274.37


The profit over here is just an hypothetical number which is a <b>positive number</b>, in real life it could be a <b>negative number</b> and if we had more data on demographic factors like age, location, gender etc. then we can dive deep onto how to profile different persona in order to target them

# <center><b>Q2:</b> Disjointed Intervals coding question</center>

We are looking for a program that manages disjointed intervals of integers. <br>
E.g.: [[1, 3], [4, 6]] is a valid object gives two intervals. <br>
[[1, 3], [3, 6]] is not a valid object because it is not disjoint. <br>
[[1, 6]] is the intended result.<br>
Empty array [] means no interval, it is the default/start state. <br>
We want you to implement two functions:<br>
add(from, to) remove(from, to)<br>
Here is an example sequence:<br>
Start: []<br>
* Call: add(1, 5) => [[1, 5]]<br>
* Call: remove(2, 3) => [[1, 2], [3, 5]]<br>
* Call: add(6, 8) => [[1, 2], [3, 5], [6, 8]] <br>
* Call: remove(4, 7) => [[1, 2], [3, 4], [7, 8]] <br>
* Call: add(2, 7) => [[1, 8]]<br>


In [9]:
def isOverlap(arrA, arrB):
    return (min(arrA[1], arrB[1]) >= max(arrA[0], arrB[0]))
isOverlap([1,8], [5,6])

True

In [1]:
def add(new_array):
    temp = []
    n = len(intervals)
    print(f'N is :{n}')
    ###  case 1: starting with the initial case where the array is empty, we will return the same array that was added
    if len(intervals) == 0:
        return [new_array]
    ### Case 2: Placing the new array at the beginning or at the end;
    if (new_array[1] < intervals[0][0] or new_array[0]> intervals[n-1][1]):
        ### case 2a: Placing the array at the front;
        if (new_array[1] < intervals[0][0]):
            temp.append(new_array)
            for i in intervals:
                temp.append(i)
            return temp
        ### case 2b: Placing the new array at the end of the existing array;
        if (new_array[0]> intervals[n-1][1]):
            for i in intervals:
                temp.append(i)
            temp.append(new_array)
            return temp
        ### case 3: Encapsualting the old array completely given that new array covers all the elements;
    if (new_array[0]<= intervals[0][0] and new_array[1] >= intervals[n-1][1]):
        temp = new_array
        return temp
    ### Overlap cases:
    overlap = True
    temp2 = []
    temp3 = [-100,-100]
    for i in range(n):
        overlap = isOverlap(intervals[i],new_array)
        if overlap == False:
            ###In case of no overlap, finding the position where new array can be inserted;
            if new_array[0]>intervals[i][1] and new_array[1]<intervals[i+1][0]:
#                 temp2.append(intervals[i])
                temp = intervals
                temp.insert(i+1, new_array)
                return temp
            if temp3 != [-100, -100]:   
                 if temp3[1]<intervals[i][0]:
                        temp2.insert(i+1,intervals[i])
#                         temp2.append(intervals[i])
        
#         temp3.insert(0,min(new_array[0], intervals[i][0]))
        temp3[0] = min(new_array[0], intervals[i][0])
        #print(f'temp2:{temp2}')
        while i<n and overlap == True:
            print(f'loop start:{i}')
            temp3[1] = max(new_array[1], intervals[i][1])
#             temp3.insert(1, max(new_array[1], intervals[i][1]))
            print(f' temp3 is {temp3}')
            if i==(n-1):
                overlap = False
            else:
                overlap = isOverlap(intervals[i+1],new_array)
            print(f'loop end{i}')
            i = i+1
        
            #print(f'iaa:::{i}')
        i = i-1
    temp2.append(temp3)
        
    return temp2


In [39]:
def remove(array):
    temp = []
    n = len(intervals)
    overlap = True
    for i in range(n):
        overlap = isOverlap(array, intervals[i])
        #### case1: if not overlap
        if overlap == False:
            ### Add this interval in temp2 directly

            temp1 = [[]]
            while i<n and overlap == True:
                ###interval lies within the array;
                if array[0]>intervals[i][0] and array[1]<intervals[i][1]:
                    temp1.insert([0][0], intervals[i][0])
                    temp1.insert([0][1], array[0])
                    temp1.insert([1][0], intervals[i][1])
                    temp1.insert([1][1], array[1])
                ### 2 sets intersects each other
                else: 
                    temp1.insert([0][0], min(array[0], intervals[i][0]))
                    temp1.insert([0][1], max(array[0], intervals[i][0]))
                    temp1.insert([1][0], min(array[1], intervals[i][1]))
                    temp1.insert([1][1], max(array[1], intervals[i][1]))
                    
                ###Check if second element of temp1 overlaps with next element
                next_overlap = isOverlap(temp1[1], array)
                if next_overlap == True:
                    temp1.insert([1][0], min(array[1], intervals[i][1]))
                    temp1.insert([1][1], max(array[1], intervals[i][1]))
                i = i+1
            i = i-1
                    

In [4]:
intervals = []

In [5]:
intervals = add([15,30])
intervals

N is :0


[[15, 30]]

In [6]:
intervals = add([1,3])
intervals

N is :1


[[1, 3], [15, 30]]

In [7]:
intervals = add([31,35])
intervals

N is :2


[[1, 3], [15, 30], [31, 35]]

In [10]:
intervals = add([4,5])
intervals

N is :3


[[1, 3], [4, 5], [15, 30], [31, 35]]

In [11]:
intervals = add([6,12])
intervals

N is :4


[[1, 3], [4, 5], [6, 12], [15, 30], [31, 35]]

In [12]:
intervals = add([7,16])
intervals

N is :5
loop start:2
 temp3 is [6, 16]
loop end2
loop start:3
 temp3 is [6, 30]
loop end3
loop start:3
 temp3 is [7, 30]
loop end3


[[4, 5], [31, 35], [7, 30]]